In [1]:
from konlpy.tag import Kkma, Hannanum, Komoran, Mecab, Twitter
import sentencepiece as spm
import re
import string
from bs4 import BeautifulSoup
import urllib.request
import feedparser
import requests
from collections import defaultdict
from operator import itemgetter
from tqdm.notebook import tqdm
from itertools import product
import pandas as pd
import pickle

In [10]:
with open('dc_LOL_newWord.pkl', 'rb') as f:
    words = pickle.load(f)
with open('DC_LOL_preprocessed.pkl', 'rb') as f:
    sents = pickle.load(f)

In [22]:
def recover(tokens):
    sent = ''.join(tokens)
    sent = sent.replace('▁', ' ')
    return sent[1:]

def example_of_word (sents, words, idx = None, by_word = None, N = 15):
    if by_word is not None:
        word = by_word
        print(f'Target word: {word}')
        print(f'Word in new word list: {word in words}')
    elif idx is not None:
        word = words[idx]
        print(f'Target word: {word}')
    
    answer = []
    cowords = defaultdict(int)
    for sent in sents:
        comparable = list(map(lambda x: x.replace('▁', ''), sent))
        if word in comparable:
            answer.append(recover(sent))
            for w in comparable:
                cowords[w] += 1
    print(f'TOTAL SENTENCES: {len(answer)}')
    sorted_cowords = list(map(lambda x: x[0], sorted(cowords.items(), key = itemgetter(1), reverse = True)))
    new_cowords = {w : cowords[w] for w in sorted_cowords if w in words and w != word}
    TopN = dict(sorted(new_cowords.items(), key=itemgetter(1), reverse=True)[:N])
    print(f'TOP {N} NEAR NEW WORDS: {TopN}')
    return answer

In [23]:
result = example_of_word(sents, words, by_word = '페이커')

Target word: 페이커
Word in new word list: True
TOTAL SENTENCES: 12227
TOP 15 NEAR NEW WORDS: {'쵸비': 690, '슼갈': 407, '는거': 396, '롤갤': 275, 'lck': 262, '더샤이': 253, '라인전': 218, '도인비': 203, '페독': 194, '는건': 190, '솔킬': 178, '슼갈들': 173, '념글': 163, 'skt': 161, '쵸비가': 146}


In [84]:
with open('senti_info.json', encoding='utf-8-sig', mode='r') as f:
    data = json.load(f)
    
def word_polarity(wordname):
    result = None
    if data.get(wordname):
        return int(data[wordname])
    return result

from collections import Counter
def sent_with_new_word (sents, word):
    '''
    sents: 문장 모음.
    word: 타겟으로 하는 신조어
    '''
    answer = []
    for sent in sents:
        comparable = list(map(lambda x: x.replace('▁', ''), sent))
        if word in comparable:
            answer += comparable
    return Counter(answer)

def new_word_sentiment (word):
    point_sum = []
    cnt = 0
    samples = sent_with_new_word(sents, word)
    for w in list(samples):
        if w == word:
            continue
        point = word_polarity(w)
        if point is not None:
            point_sum.append(point * samples[w])
            cnt += samples[w]
                
    return sum(point_sum) / cnt
                

In [85]:
scores = {}
for word in tqdm(words):
    scores[word] = new_word_sentiment(word)
    tqdm.write('{}: {}'.format(word, scores[word]))


보이노: -0.42660550458715596
롤갤: -0.5318632855567806
씹덕: -0.3438735177865613
는거: -0.2908909574468085
개웃기네: -0.5517241379310345
DRX: -0.1629249518922386
도못: -0.42857142857142855
좆망: -0.14874141876430205
비씹덕: -0.3763440860215054
슼갈새끼들: -0.6696165191740413
월클: -0.5928571428571429
좆으로: -0.07446808510638298
넛신: -0.12658227848101267
챌코: -0.059490084985835696
쳐먹: -0.815592203898051
개병신: -0.5194244604316547
좆병신: -0.6295546558704453
노ᄏᄏ: -0.28760162601626016
더샤이: 0.012515644555694618
첫짤: -0.3
씨맥이랑: -0.7768361581920904
skt: -0.08072372999304106
부두술: 1.0586907449209932
슼한테: -0.5747126436781609
op: -0.21705426356589147
슼갈: -0.5176470588235295
네ᄏᄏ: -0.3148558758314856
대깨맥: -0.41895966830003767
상관없: -0.3832752613240418
욕처먹: -0.5581395348837209
틀딱: -0.3154269972451791
탑솔러: 0.25882352941176473
애미창년: -0.7379912663755459
재밌: -0.16142270861833105
잘생겼: 0.24528301886792453
젠첩: -0.3290653008962868
씨맥: -0.4733199093883715
APK: 0.0
RNG: -0.18041237113402062
주작하네: -0.3684210526315789
올라왔: -0.14285714285714285
했누:

줘패: -0.35294117647058826
씹호감: -0.031746031746031744
겠누: -0.08085106382978724
피싸개: -0.4540763673890609
한최탑: -0.3333333333333333
고싶다: -0.43951612903225806
씹창: -0.4494949494949495
갱플: -0.34913793103448276
ck: -0.2702702702702703
미쳣: 0.0
황드: 0.08636363636363636
개씨발: -0.4574780058651026
많았: -0.056530214424951264
지말자: -0.281767955801105
대퍼팀: 0.2231404958677686
탑정글: -0.030042918454935622
역체원: 0.33548387096774196
련들: -0.5748299319727891
lpl: -0.12975098296199214
개노잼: -0.2919254658385093
롤붕이: -0.16359447004608296
대깨: -0.36220472440944884
슼갈은: -0.6136986301369863
었냐: -0.3316582914572864
같노: -0.569811320754717
스멥: 0.22137404580152673
벤픽: -0.08968609865470852
쇼메이커: 0.043478260869565216
프차: -0.2950310559006211
올렸: -0.45544554455445546
cj: -0.24603174603174602
시발련아: -0.7867298578199052
개념글: -0.4593406593406593
들한테: -0.24158125915080528
시발아: -0.38596491228070173
FPX: -0.14678899082568808
역체미: 0.19736842105263158
닮았: -0.14563106796116504
킅갈: -0.41304347826086957
존마드: -0.44329896907216493
인섹: -0.180180

ZeroDivisionError: division by zero

In [88]:
for word in tqdm(words[479:]):
    try:
        scores[word] = new_word_sentiment(word)
        tqdm.write('{}: {}'.format(word, scores[word]))
    except:
        continue

킹쵸비: -0.14634146341463414
노ᄏᄏᄏᄏ: -0.2608695652173913
이도준: 0.5
=1: -0.14285714285714285
싺싺: -2.0
꺆꺆: 0.3333333333333333
https: -0.4
놐ᄏᄏᄏ: -0.2222222222222222
킼킼: 2.0
뾱뾱: 1.0
쒹쒹: -1.0
쭵쭵: -2.0
=2: 1.0
랎콦: -2.0
EB: 2.0
쀏쒧: -2.0
leagueoflegends: -2.0
슉슉: -0.5
뀽뀽: -2.0
큭큭큭: -1.0
켁켁켁: 0.0
youtube: -0.6666666666666666
끙끙앓: -1.5
샅샅: -0.5
맜춥뻡: 1.0
궷슘: -2.0
3&: -2.0
힉힠: 0.0
끆끆: -1.0
콱큠: 0.0
꽊꽊: 1.0
헠헠: 2.0
꾿꾿: 0.0
솞듮: 0.0
댼됵: 0.0
쩨됟: -2.0
뀍뀍: 0.0
페이커: -0.09477164197379699

